### Run this in Python kernel

In [ ]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Learn Golden Cross before this notebook

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.portfolio :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure-backtesting.large-data :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [9]:
(load-large-dataset "../resources/CRSP-extract-sorted.csv" "main" add-aprc-by-date)
(set-main "main")

true

### Initialise portfolio （Go back here everytime you want to restart.）

In [10]:
;; initialise with current date and initial capital (= $10000)
(init-portfolio "1980-12-15" 1000);

### Write a strategy

The Bollinger Bands indicator is used to measure the “highness” or “lowness” of the price, relative to previous trades.
The Bollinger Bands consist of a middle band with two outer bands:

Middle Band=20-day simple moving average (SMA)

Upper Band=20-day SMA+(20-day standard deviation of price×2)

Lower Band=20-day SMA−(20-day standard deviation of price×2)

Buy signal: when price goes below lower band

Sell signal: when price goes above upper band


large dataset

In [17]:
(get-date)

"2014-12-29"

In [5]:
(def MA20-vecs (atom {}))
(defn bollinger-bands
    [tic]
    "The strategy"
    (if (not (get (deref MA20-vecs) tic))
        (swap! MA20-vecs assoc tic []))
    (swap! MA20-vecs assoc tic (get-prev-n-days :PRC 20 tic (get (deref MA20-vecs) tic)))
      (let [MA20-vec (get (deref MA20-vecs) tic)
            price (get-price tic) 
            SMA (moving-average :PRC MA20-vec) 
            SD20 (moving-sd :PRC MA20-vec) 
            [upper lower] [(+ SMA SD20) (- SMA SD20)]]
        (if (< price lower)
          (order-lazy tic 0.1 :print false))
          (if (> price upper)
              (order-lazy tic 0 :remaining true))))

#'clojure-backtesting.demo/bollinger-bands

In [11]:
(def MA20-vec [])
(while (< (compare (get-date) "2014-12-26") 0)
    (do
      (bollinger-bands "AAPL")
      (bollinger-bands "F"))
      (update-eval-report (get-date))
      (next-date))
(end-order)

nil

In [22]:
;(pprint/print-table (take 200 (deref portfolio-value)))
(take-last 2  (deref order-record))

({:date "2014-12-29", :tic "AAPL", :price 113.91, :aprc "295.01", :quantity -0.6} {:date "2014-12-29", :tic "F", :price 15.52, :aprc "622.70", :quantity -0.4})

### Check portfolio record

In [14]:
;; view final portfolio
(view-portfolio)


| :asset | :price |    :aprc | :quantity | :tot-val |
|--------+--------+----------+-----------+----------|
|   cash |    N/A |      N/A |       N/A |   458.39 |
|   AAPL | 113.91 | 295.0143 |       0.6 |   177.01 |
|      F |  15.52 | 622.6971 |       0.4 |   249.08 |


nil

In [15]:
;; view portfolio value and return
(view-portfolio-record 10)


|      :date | :tot-value | :daily-ret | :tot-ret | :loan | :leverage | :margin |
|------------+------------+------------+----------+-------+-----------+---------|
| 1980-12-15 |   $1000.00 |      0.00% |    0.00% | $0.00 |      0.00 |   0.00% |
| 1980-12-16 |   $1000.00 |      0.00% |    0.00% | $0.00 |      0.00 |   0.00% |
| 1980-12-17 |   $1000.03 |      0.00% |    0.00% | $0.00 |      0.00 |   0.00% |
| 1980-12-18 |   $1000.09 |      0.00% |    0.00% | $0.00 |      0.00 |   0.00% |
| 1980-12-19 |   $1000.29 |      0.00% |    0.01% | $0.00 |      0.00 |   0.00% |
| 1980-12-22 |   $1000.74 |      0.01% |    0.03% | $0.00 |      0.00 |   0.00% |
| 1980-12-23 |   $1000.98 |      0.00% |    0.04% | $0.00 |      0.00 |   0.00% |
| 1980-12-24 |   $1001.35 |      0.00% |    0.06% | $0.00 |      0.00 |   0.00% |
| 1980-12-26 |   $1002.12 |      0.00% |    0.09% | $0.00 |      0.00 |   0.00% |
| 1980-12-29 |   $1002.26 |      0.01% |    0.10% | $0.00 |      0.00 |   0.00% |


nil

### Generate evaluation report

In [7]:
(eval-report 434)
;; (update-eval-report)
;; (count(deref eval-report-data))
;; (deref dailyret-record)


|      :date | :tot-value |     :vol |                 :sharpe | :pnl-pt |
|------------+------------+----------+-------------------------+---------|
| 1981-02-26 |      $1000 |  0.0000% |                 0.0000% |      $0 |
| 1981-02-27 |      $1001 |  0.0285% |                 1.7362% |      $0 |
| 1981-03-02 |      $1001 |  0.0122% |                 5.9335% |      $0 |
| 1981-03-03 |      $1001 |  0.0000% |                 0.0000% |      $0 |
| 1981-03-04 |      $1000 |  0.0199% |                 1.0642% |      $0 |
| 1981-03-05 |      $1011 |  0.1814% |                 2.7529% |      $0 |
| 1981-03-06 |      $1028 |  0.2676% |                 4.6055% |      $2 |
| 1981-03-09 |      $1017 |  0.1588% |                 4.6637% |      $1 |
| 1981-03-10 |      $1019 |  0.0291% |                28.1887% |      $1 |
| 1981-03-11 |      $1026 |  0.0059% |               142.8007% |      $1 |
| 1981-03-12 |      $1028 |  0.0055% |               155.8741% |      $1 |
| 1981-03-13 |      $103

| 1981-07-31 |       $801 |  2.0880% |               -84.4380% |     $-1 |
| 1981-08-03 |       $832 |  2.0398% |               -75.6972% |     $-1 |
| 1981-08-04 |       $712 | 10.2937% |               -25.4924% |     $-1 |
| 1981-08-05 |       $712 |  0.0000% |                 0.0000% |     $-1 |
| 1981-08-06 |       $681 |  3.0895% |               -95.6151% |     $-1 |
| 1981-08-07 |       $618 |  7.4469% |               -50.3913% |     $-2 |
| 1981-08-10 |       $586 |  4.3140% |               -97.7555% |     $-2 |
| 1981-08-11 |       $618 |  4.1437% |               -90.9563% |     $-2 |
| 1981-08-12 |       $683 |  7.0038% |               -42.9506% |     $-1 |
| 1981-08-13 |       $683 |  0.0000% |                 0.0000% |     $-1 |
| 1981-08-14 |       $650 |  3.6785% |               -92.7320% |     $-2 |
| 1981-08-17 |       $583 |  9.0280% |               -48.7839% |     $-2 |
| 1981-08-18 |       $583 |  0.0000% | -1689988096198580.0000% |     $-2 |
| 1981-08-19 |       $549

| 1982-01-07 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-08 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-11 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-12 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-13 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-14 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-15 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-18 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-19 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-20 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-21 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-22 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-25 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-26 |       $446

| 1982-06-15 |       $484 | 19.7274% |               -48.5475% |     $-2 |
| 1982-06-16 |       $484 |  7.7769% |              -123.1477% |     $-2 |
| 1982-06-17 |       $470 |  8.5562% |              -117.7430% |     $-2 |
| 1982-06-18 |       $456 |  8.6795% |              -122.3232% |     $-2 |
| 1982-06-21 |       $498 | 13.3145% |               -68.6373% |     $-2 |
| 1982-06-22 |       $640 | 24.9738% |               -23.5715% |     $-1 |
| 1982-06-23 |       $626 |  7.9683% |               -77.6529% |     $-1 |
| 1982-06-24 |       $538 | 18.7649% |               -45.5899% |     $-1 |
| 1982-06-25 |       $568 |  9.6425% |               -80.3339% |     $-1 |
| 1982-06-28 |       $598 |  9.3263% |               -75.1125% |     $-1 |
| 1982-06-29 |       $598 |  7.5935% |               -92.2536% |     $-1 |
| 1982-06-30 |       $504 | 22.4368% |               -44.3573% |     $-2 |
| 1982-07-01 |       $373 | 50.6630% |               -33.4631% |     $-2 |
| 1982-07-02 |       $356

nil

### Plot variables

In [ ]:
(def data (deref portfolio-value))

In [ ]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

In [ ]:
(first data-to-plot)

In [ ]:
(plot data-to-plot :plot :date :daily-ret)